# MODNet 'matbench_expt_gap' benchmarking

The `matbench_expt_gap` dataset contains measured band gaps for 4604 compositions of inorganic semiconductors from Zhuo *et al.*, JPCL.

In [ ]:
from collections import defaultdict
import itertools
import os
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
from IPython.display import Markdown
from matminer.datasets import load_dataset, get_all_dataset_info
from pymatgen.core import Composition

from modnet.preprocessing import MODData
from modnet.featurizers import MODFeaturizer
from modnet.featurizers.presets import DeBreuck2020Featurizer

os.environ["CUDA_VISIBLE_DEVICES"] = "5"

In [ ]:
Markdown(filename="./README.md")

## Data exploration

In [ ]:
df = load_dataset("matbench_expt_gap")
df["composition"] = df["composition"].map(Composition)

### Target space

In [ ]:
df.describe()

In [ ]:
fig, ax = plt.subplots(facecolor="w")
ax.hist(df.where(df["gap expt"] == 0)["gap expt"], bins=1, density=False, label="Zero band gap")
ax.hist(df.where(df["gap expt"] > 0)["gap expt"], bins=11, density=False, label="Non-zero band gap")
ax.set_ylabel("Frequency")
ax.set_xlabel("Band gap (eV)")
ax.legend()

## Featurization and feature selection

First, we define some convenience classes that pass wraps composition data in a fake structure containe, and we define a composition only featurizer preset based on `DeBreuck2020Featurizer`.

In [ ]:
class CompositionOnlyFeaturizer(MODFeaturizer):
    composition_featurizers = DeBreuck2020Featurizer.composition_featurizers
    
    def featurize_composition(self, df):
        """ Applies the preset composition featurizers to the input dataframe,
        renames some fields and cleans the output dataframe.

        """
        from pymatgen.core.periodic_table import Element 
        import numpy as np
        from modnet.featurizers import clean_df
        df = super().featurize_composition(df)
        _orbitals = {"s": 1, "p": 2, "d": 3, "f": 4}
        df['AtomicOrbitals|HOMO_character'] = df['AtomicOrbitals|HOMO_character'].map(_orbitals)
        df['AtomicOrbitals|LUMO_character'] = df['AtomicOrbitals|LUMO_character'].map(_orbitals)

        df['AtomicOrbitals|HOMO_element'] = df['AtomicOrbitals|HOMO_element'].apply(
            lambda x: -1 if not isinstance(x, str) else Element(x).Z
        )
        df['AtomicOrbitals|LUMO_element'] = df['AtomicOrbitals|LUMO_element'].apply(
            lambda x: -1 if not isinstance(x, str) else Element(x).Z
        )

        df = df.replace([np.inf, -np.inf, np.nan], 0)
        
        return clean_df(df)

class CompositionContainer:
    def __init__(self, composition):
        self.composition = composition

In [ ]:
PRECOMPUTED_MODDATA = "./precomputed/expt_gap_benchmark_moddata.pkl.gz"

if os.path.isfile(PRECOMPUTED_MODDATA):
    data = MODData.load(PRECOMPUTED_MODDATA)
else:
    # Use a fresh copy of the dataset
    df = load_dataset("matbench_expt_gap")
    df["composition"] = df["composition"].map(Composition)
    df["structure"] = df["composition"].map(CompositionContainer)
    
    data = MODData(
        structures=df["structure"].tolist(), 
        targets=df["E_g"].tolist(), 
        target_names=["gap expt (eV)"],
        featurizer=CompositionOnlyFeaturizer(n_jobs=8)
    )
    data.featurize()
    data.feature_selection(n=-1)

In [ ]:
#data.optimal_features=None
#data.cross_nmi = None
#data.num_classes = {"E_g":0}
#data.feature_selection(n=-1)
#data.save("./precomputed/expt_gap_benchmark_moddata_MPCNMI.pkl.gz")

## Training

In [ ]:
try:
    plot_benchmark
except:
    import sys
    sys.path.append('..')
    from modnet_matbench.utils import *
    
from sklearn.model_selection import KFold
from modnet.models import MODNetModel
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

best_settings = {
    "increase_bs":False,
    "num_neurons": [[256], [128], [16], [16]],
    "n_feat": 100,
    "lr": 0.007,
    "epochs": 400,
    "verbose": 0,
    "act": "elu",
    "batch_size": 64,
    "loss": "mae",
}

results = matbench_benchmark(data, [[["E_g"]]], {"E_g": 1}, best_settings, save_folds=True)
np.mean(results['scores'])

In [ ]:
best_settings = {
    "increase_bs":False,
    "num_neurons": [[256], [128], [16], [16]],
    "n_feat": 100,
    "lr": 0.007,
    "epochs": 400,
    "verbose": 0,
    "act": "elu",
    "batch_size": 64,
    "loss": "mae",
} #0.31

In [ ]:
for i in range(5):
    plt.plot(results["models"][i].history.history["loss"][10:])

In [ ]:
import seaborn as sns
reg_df = pd.DataFrame(
    np.array([
        [x for targ in results["targets"] for x in targ],
        [y for pred in results["predictions"] for y in pred],
        [e for err in results["errors"] for e in err]
    ]).T,
    columns=["targets", "predictions", "errors"]
)
splits = []
for i in range(5):
    for j in range(len(results["targets"][i])):
        splits.append(i)
reg_df["split"] = splits

In [ ]:
fig, ax = plt.subplots()
#ax.set_aspect("equal")
sns.scatterplot(data=reg_df, x="targets", y="predictions", hue="split", palette="Dark2", ax=ax, alpha=0.2)
sns.regplot(data=reg_df, x="targets", y="predictions", ax=ax, scatter=False)
# plt.plot(*ax.get_xlim(), *ax.get_xlim(), c="k",alpha=0.5)
ax.set_xlim(-1)
plt.xlabel("True")
plt.ylabel("Pred.")

In [ ]:
g = sns.jointplot(data=reg_df, x="errors", y="targets", hue="split", palette="Dark2", alpha=0.0, marginal_kws={"shade": False})
g.plot_joint(sns.scatterplot, hue=None, c="black", s=5, alpha=0.5)
g.plot_joint(sns.kdeplot, color="split", zorder=0, levels=5, alpha=0.5)